In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer


try:
    from xgboost import XGBRegressor
except Exception:
    XGBRegressor = None
try:
    from lightgbm import LGBMRegressor
except Exception:
    LGBMRegressor = None
    
    
try:
    from statsmodels.tsa.arima.model import ARIMA
    from statsmodels.tsa.statespace.sarimax import SARIMAX
except Exception:
    ARIMA = SARIMAX = None

In [2]:
sns.set(style="whitegrid")
np.random.seed(42)

folder_tag="_lag" # modify path if lag features are used / planned to be used when lag features are not used


model_plot_path=f"artifacts/model_plots{folder_tag}/"
model_path=f"artifacts/models{folder_tag}/"
model_results_path=f"artifacts/model_results{folder_tag}/"

# Create folders for artifacts & models
os.makedirs(model_plot_path, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(model_results_path, exist_ok=True)

In [3]:
INPUT_CSV = "artifacts/data/clean_data_with_lag_roll.csv"  # change as needed

# Target variable name (ensure matches csv)
TARGET = "Average_Price"

# Which models to run (strings): "Linear", "RandomForest", "XGBoost", "LightGBM", "ARIMA"
RUN_MODELS = ["Linear", "RandomForest", "XGBoost", "LightGBM", "ARIMA"]

# TimeSeriesSplit folds
N_SPLITS = 5

# CV scoring metrics list (modify as needed)
# Will be used for final reporting. Keep names consistent with regression_metrics below.
METRIC_NAMES = ["RMSE", "MSE", "MAE", "MAPE", "R2"]

# Randomized / Grid search settings
RANDOM_SEARCH_ITER = 20
GRID_SEARCH_SMALL = True  # if True, run smaller grids to save time

# Use n_jobs=1 to avoid multiprocessing issues in constrained environments
# N_JOBS = 1

In [4]:
def regression_metrics(y_true, y_pred):
    """Return dictionary of metrics for numeric arrays/series."""
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    # safe MAPE
    mask = y_true != 0
    mape = np.nan
    if mask.sum() > 0:
        mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    r2 = r2_score(y_true, y_pred)
    return {"RMSE": rmse, "MSE": mse, "MAE": mae, "MAPE": mape, "R2": r2}


In [5]:
# sklearn scorers (note: sklearn expects higher-is-better)
sk_rmse = make_scorer(lambda y, yhat: -np.sqrt(mean_squared_error(y, yhat)))  # negative RMSE
sk_mse  = make_scorer(lambda y, yhat: -mean_squared_error(y, yhat))
sk_mae  = make_scorer(lambda y, yhat: -mean_absolute_error(y, yhat))
def sk_mape(y, yhat):
    mask = y != 0
    if mask.sum()==0:
        return 0.0
    return -np.mean(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100
sk_mape_scorer = make_scorer(sk_mape)


In [6]:
# map names to scorers for GridSearchCV
SCORING = {
    "neg_rmse": sk_rmse,
    "neg_mse": sk_mse,
    "neg_mae": sk_mae,
    "neg_mape": sk_mape_scorer,
    "r2": "r2"
}

In [ ]:
def save_plot(fig, name):
    filepath = os.path.join(model_plot_path, f"{name}.png")
    fig.savefig(filepath, bbox_inches="tight", dpi=200)
    plt.close(fig)
    print("Saved:", filepath)


In [8]:
# --------- 3. Load data ---------
df = pd.read_csv(INPUT_CSV, parse_dates=["Date"], infer_datetime_format=True)
print("Loaded:", INPUT_CSV, " shape:", df.shape)

# Quick drop rows with missing target
df = df.dropna(subset=[TARGET]).reset_index(drop=True)

# Separate features & target, keep Date for potential time-splits/plots
if "Date" in df.columns:
    df = df.sort_values("Date").reset_index(drop=True)


Loaded: artifacts/data/clean_data_with_lag_roll.csv  shape: (1300, 35)


In [9]:
df_no_lag = df.copy()


In [10]:
cols_all = [c for c in df.columns if c not in [TARGET]]

# # Exclude direct leakage: if any column equal to TARGET or 'imported_tomato_price' recorded same time, you can adjust here
# if "imported_tomato_price" in cols_all:
#     # drop from modeling if same-day leak for forecasting; user insisted earlier that it's leakage for forecasting
#     cols_all.remove("imported_tomato_price")
#     print("Removed 'imported_tomato_price' from features (contemporaneous leak).")

# Drop Date from features
if "Date" in cols_all:
    cols_all.remove("Date")

In [11]:
# Identify numeric and categorical automatically
numeric_cols = df[cols_all].select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in cols_all if c not in numeric_cols]

print("NUMERIC cols:", numeric_cols)
print("CATEGORICAL cols:", cat_cols)


NUMERIC cols: ['Supply_Volume', 'Temperature', 'Precipitation', 'Wind_Speed', 'Air_Pressure', 'Rainfall_MM', 'USD_TO_NPR', 'Diesel', 'is_festival', 'imported_tomato_price', 'Inflation', 'day', 'month', 'day_of_week', 'is_weekend', 'month_sin', 'month_cos', 'Average_Price_lag1', 'Average_Price_lag7', 'Average_Price_lag30', 'Average_Price_rollmean_7', 'Average_Price_rollmean_30', 'Average_Price_rollmean_90', 'Supply_Volume_rollmean_7', 'Supply_Volume_rollmean_30', 'Supply_Volume_rollmean_90', 'imported_tomato_price_rollmean_7', 'imported_tomato_price_rollmean_30', 'imported_tomato_price_rollmean_90']
CATEGORICAL cols: ['Season_Autumn', 'Season_Monsoon', 'Season_Spring', 'Season_Winter']


In [12]:
# For no-lag experiment, optionally drop lag columns (heuristic: columns with '_lag' or '_roll' in name)
def filter_no_lag_columns(cols):
    return [c for c in cols if (("_lag" not in c) and ("_roll" not in c) and ("lag_" not in c) and ("roll" not in c))]


In [13]:
# Create feature sets
FEATURES_FULL = cols_all                                    # all features present in CSV
FEATURES_NO_LAG = filter_no_lag_columns(FEATURES_FULL)      # remove lag/roll columns for no-lag model

print("FEATURES_FULL count:", len(FEATURES_FULL))
print("FEATURES_NO_LAG count:", len(FEATURES_NO_LAG))


FEATURES_FULL count: 33
FEATURES_NO_LAG count: 21


In [14]:
# Numeric pipeline: impute (ffill is already used earlier but we still include a safe imputer) + scaling
num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),   # safe; data should already be filled but keep for robustness
    ("scaler", StandardScaler())
])

# Categorical pipeline: impute -> one-hot (if any categories)
cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, numeric_cols),
    ("cat", cat_pipeline, cat_cols)
], remainder="drop", sparse_threshold=0)

# Helper to build a full sklearn Pipeline for a given estimator
def make_pipeline(estimator):
    return Pipeline([
        ("preproc", preprocessor),
        ("est", estimator)
    ])

In [ ]:
models = {}
param_grids = {}

# 6.1 Linear
models["Linear"] = make_pipeline(LinearRegression())
param_grids["Linear"] = {
    # Linear has no hyperparams here; include Ridge variant via separate model if needed
}

# 6.2 RandomForest
models["RandomForest"] = make_pipeline(RandomForestRegressor(random_state=42))
param_grids["RandomForest"] = {
    "est__n_estimators": [100, 200] if GRID_SEARCH_SMALL else [100, 200, 400],
    "est__max_depth": [5, 10, None],
    "est__min_samples_leaf": [1, 2, 4]
}

# 6.3 XGBoost (if available)
if XGBRegressor is not None:
    models["XGBoost"] = make_pipeline(XGBRegressor(objective="reg:squarederror", random_state=42))
    param_grids["XGBoost"] = {
        "est__n_estimators": [100, 200],
        "est__max_depth": [3, 5],
        "est__learning_rate": [0.01, 0.05, 0.1]
    }
else:
    print("XGBoost not available in this environment; skipping XGBoost.")

# 6.4 LightGBM (if available)
if LGBMRegressor is not None:
    models["LightGBM"] = make_pipeline(LGBMRegressor(random_state=42))
    param_grids["LightGBM"] = {
        "est__n_estimators": [100, 200],
        "est__num_leaves": [31, 64],
        "est__learning_rate": [0.01, 0.05]
    }
else:
    print("LightGBM not available in this environment; skipping LightGBM.")

# 6.5 ARIMA/SARIMA handled separately (not inside sklearn pipeline)
if ARIMA is None:
    print("statsmodels ARIMA/SARIMAX not available; ARIMA steps will be skipped.")

# --------- 7. Train/validate with TimeSeriesSplit + hyperparameter tuning ---------
# Choose which feature set to use: full (with lags) or no-lag
USE_LAG_FEATURES = True  # change to False to force 'no-lag' experiment
FEATURES_TO_USE = FEATURES_FULL if USE_LAG_FEATURES else FEATURES_NO_LAG
print("Using features count:", len(FEATURES_TO_USE))

X = df[FEATURES_TO_USE].copy()
y = df[TARGET].copy()

bool_cols = X.select_dtypes(include=["bool"]).columns
X[bool_cols] = X[bool_cols].astype(int)

# Ensure no NA in X due to feature selection; imputer in pipeline handles remaining NAs.
print("X shape:", X.shape, " y shape:", y.shape)

tscv = TimeSeriesSplit(n_splits=N_SPLITS)

# Storage for results
cv_summary = []
best_models = {}

# Iterate models for training and hyperparameter tuning
for name, pipeline in models.items():
    if name not in RUN_MODELS:
        continue
    print("\n" + "="*40)
    print("Training model:", name)
    print("="*40)

    grid = param_grids.get(name, None)

    # If no grid (e.g., Linear), just cross-validate without search
    if not grid:
        print("No hyperparameter grid for", name, " — running cross_validate with TimeSeriesSplit.")
        cv_res = cross_validate(
        pipeline, X, y,
        cv=tscv,
        scoring=SCORING,
        return_train_score=True
        )

        # Compute average metrics from cv_res (note neg scorers are negative)
        # Convert negative scorers to positive metrics
        results = {
            "Model": name,
            "RMSE_mean": -np.mean(cv_res["test_neg_rmse"]) if "test_neg_rmse" in cv_res else np.nan,
            "MSE_mean": -np.mean(cv_res["test_neg_mse"]) if "test_neg_mse" in cv_res else np.nan,
            "MAE_mean": -np.mean(cv_res["test_neg_mae"]) if "test_neg_mae" in cv_res else np.nan,
            "MAPE_mean": -np.mean(cv_res["test_neg_mape"]) if "test_neg_mape" in cv_res else np.nan,
            "R2_mean": np.mean(cv_res["test_r2"]) if "test_r2" in cv_res else np.nan
        }
        cv_summary.append(results)

        # Fit on full training portion (we will treat the last 20% as test later)
        fitted = pipeline.fit(X, y)
        best_models[name] = fitted
        # Save fitted model
        joblib.dump(fitted, f"{model_path}{name}_fitted.joblib")
        print("Saved model:", f"{model_path}{name}_fitted.joblib")
        continue

    # If grid provided -> run RandomizedSearch then refine with GridSearch (optional)
    # Randomized Search (broad)
    print("Running RandomizedSearchCV (broad) for", name)
    rnd = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=grid,
        n_iter=RANDOM_SEARCH_ITER,
        cv=tscv,
        scoring="neg_mean_squared_error",   # use neg MSE for search ranking
        random_state=42,
        # n_jobs=N_JOBS,
        verbose=1
    )
    rnd.fit(X, y)
    print("RandomSearch best params:", rnd.best_params_, " best_score:", rnd.best_score_)

    # Optionally run a smaller GridSearch around the best params (if desired)
    # We will attempt a small grid: replace the param with the found best if not present
    if GRID_SEARCH_SMALL:
        # build a small grid based on rnd.best_params_ if possible
        small_grid = {}
        for k, v in grid.items():
            if k in rnd.best_params_:
                # if the best param is inside the list of grid values, pick neighbors or keep list small
                small_choices = grid[k]
                small_grid[k] = small_choices if len(small_choices) <= 3 else small_choices[:3]
            else:
                small_grid[k] = grid[k] if isinstance(grid[k], list) else [grid[k]]
        print("Running GridSearchCV (refined) for", name)
        gscv = GridSearchCV(
            estimator=pipeline,
            param_grid=small_grid,
            cv=tscv,
            scoring="neg_mean_squared_error",
            # n_jobs=N_JOBS,
            verbose=1
        )
        gscv.fit(X, y)
        best_est = gscv.best_estimator_
        best_params = gscv.best_params_
        best_score = gscv.best_score_
        print("GridSearch best params:", best_params, "best_score:", best_score)
    else:
        best_est = rnd.best_estimator_
        best_params = rnd.best_params_
        best_score = rnd.best_score_

    # Store best estimator
    best_models[name] = best_est
    # Save model to disk
    joblib.dump(best_est, f"{model_path}{name}_best.joblib")
    print("Saved best model:", f"{model_path}{name}_best.joblib")

    # Cross-validate the best estimator to get metrics per-fold
    cv_res = cross_validate(best_est, X, y, cv=tscv, scoring=SCORING, return_train_score=False)
    results = {
        "Model": name,
        "RMSE_mean": -np.mean(cv_res["test_neg_rmse"]) if "test_neg_rmse" in cv_res else np.nan,
        "RMSE_std" : np.std([-np.mean(cv_res["test_neg_rmse"])]),
        "MSE_mean": -np.mean(cv_res["test_neg_mse"]) if "test_neg_mse" in cv_res else np.nan,
        "MAE_mean": -np.mean(cv_res["test_neg_mae"]) if "test_neg_mae" in cv_res else np.nan,
        "MAPE_mean": -np.mean(cv_res["test_neg_mape"]) if "test_neg_mape" in cv_res else np.nan,
        "R2_mean": np.mean(cv_res["test_r2"]) if "test_r2" in cv_res else np.nan
    }
    cv_summary.append(results)


Using features count: 33
X shape: (1300, 33)  y shape: (1300,)

Training model: Linear
No hyperparameter grid for Linear  — running cross_validate with TimeSeriesSplit.
Saved model: artifacts/models_lag/Linear_fitted.joblib

Training model: RandomForest
Running RandomizedSearchCV (broad) for RandomForest
Fitting 5 folds for each of 18 candidates, totalling 90 fits
RandomSearch best params: {'est__n_estimators': 100, 'est__min_samples_leaf': 4, 'est__max_depth': 5}  best_score: -60.69094302122615
Running GridSearchCV (refined) for RandomForest
Fitting 5 folds for each of 18 candidates, totalling 90 fits
GridSearch best params: {'est__max_depth': 5, 'est__min_samples_leaf': 4, 'est__n_estimators': 100} best_score: -60.69094302122615
Saved best model: artifacts/models_lag/RandomForest_best.joblib

Training model: XGBoost
Running RandomizedSearchCV (broad) for XGBoost
Fitting 5 folds for each of 12 candidates, totalling 60 fits
RandomSearch best params: {'est__n_estimators': 100, 'est__max

In [ ]:
# --- Train vs CV comparison (per model, per metric) ---

records = []

# rerun cross_validate to capture both train & test per fold
for name, pipeline in models.items():
    if name not in best_models:
        continue
    print(f"Evaluating train vs CV metrics for {name}")
    cv_res = cross_validate(
        pipeline,
        X, y,
        cv=tscv,
        scoring={
            "rmse": sk_rmse,
            "mse": sk_mse,
            "mae": sk_mae,
            "mape": sk_mape_scorer,
            "r2": "r2"
        },
        return_train_score=True
    )
    
    for metric_key in ["rmse", "mse", "mae", "mape", "r2"]:
        train_metric = cv_res.get(f"train_{metric_key}", None)
        test_metric = cv_res.get(f"test_{metric_key}", None)
        if train_metric is None or test_metric is None:
            continue

        # reverse neg sign for errors
        if "neg" in metric_key:
            train_metric = -train_metric
            test_metric = -test_metric

        for fold, (tr, ts) in enumerate(zip(train_metric, test_metric), 1):
            records.append({
                "Model": name,
                "Metric": metric_key.upper(),
                "Fold": fold,
                "Train": tr,
                "CV": ts
            })

df_comp = pd.DataFrame(records)
print(df_comp.head())


Evaluating train vs CV metrics for Linear
Evaluating train vs CV metrics for RandomForest
Evaluating train vs CV metrics for XGBoost
Evaluating train vs CV metrics for LightGBM
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000114 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1229
[LightGBM] [Info] Number of data points in the train set: 220, number of used features: 34
[LightGBM] [Info] Start training from score 72.249091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

In [ ]:
for metric in df_comp["Metric"].unique():
    fig, ax = plt.subplots(figsize=(8, 4))
    dfp = df_comp[df_comp["Metric"] == metric]
    df_melt = dfp.melt(
        id_vars=["Model", "Fold", "Metric"],
        value_vars=["Train", "CV"],
        var_name="Set",
        value_name="Score"
    )
    sns.barplot(x="Model", y="Score", hue="Set", data=df_melt, ax=ax)
    ax.set_title(f"Train vs CV — {metric}")
    ax.set_ylabel(metric)
    plt.xticks(rotation=45)
    save_plot(fig, f"train_vs_cv_{metric.lower()}_by_model")


Saved: artifacts/model_plots_lag/
Saved: artifacts/model_plots_lag/
Saved: artifacts/model_plots_lag/
Saved: artifacts/model_plots_lag/
Saved: artifacts/model_plots_lag/


In [ ]:
# We will use the last 20% of chronological data as test set (time-based holdout)
holdout_frac = 0.2
n_holdout = int(len(X) * holdout_frac)
if n_holdout < 1:
    raise RuntimeError("Dataset too small for holdout fraction.")

train_idx = slice(0, len(X) - n_holdout)
test_idx = slice(len(X) - n_holdout, len(X))

X_train_final = X.iloc[train_idx].reset_index(drop=True)
y_train_final = y.iloc[train_idx].reset_index(drop=True)
X_test_final  = X.iloc[test_idx].reset_index(drop=True)
y_test_final  = y.iloc[test_idx].reset_index(drop=True)

In [ ]:
print("\nFinal holdout sizes -> Train:", X_train_final.shape, " Test:", X_test_final.shape)


Final holdout sizes -> Train: (1040, 33)  Test: (260, 33)


In [ ]:
eval_records = []
for name, model in best_models.items():
    # Some entries may be raw pipelines already fitted or sklearn Pipelines unfit - ensure fitted
    print("\nEvaluating on holdout:", name)
    try:
        # If model is a pipeline but not fitted, fit on training portion
        if hasattr(model, "fit") and (not hasattr(model, "predict") or getattr(model, "steps", None) is None):
            model.fit(X_train_final, y_train_final)

        # Fit if not fitted (safe)
        try:
            # predict directly
            y_pred = model.predict(X_test_final)
        except Exception:
            # fit on training then predict
            model.fit(X_train_final, y_train_final)
            y_pred = model.predict(X_test_final)
    except Exception as e:
        print("Model", name, "failed on holdout:", e)
        continue

    mets = regression_metrics(y_test_final, y_pred)
    mets["Model"] = name
    eval_records.append(mets)

    # Plot actual vs predicted on holdout
    fig, ax = plt.subplots(figsize=(10,3))
    # If Date exists in df, display the last dates for x-axis
    if "Date" in df.columns:
        test_dates = df["Date"].iloc[test_idx].reset_index(drop=True)
        ax.plot(test_dates, y_test_final, label="Actual")
        ax.plot(test_dates, y_pred, linestyle="--", label="Predicted")
        ax.set_xticklabels(test_dates.dt.strftime("%Y-%m-%d"), rotation=45)
    else:
        ax.plot(y_test_final.index, y_test_final, label="Actual")
        ax.plot(y_test_final.index, y_pred, linestyle="--", label="Predicted")
    ax.set_title(f"{name} — Actual vs Predicted (Holdout)")
    ax.legend()
    plt.tight_layout()
    save_plot(fig, f"actual_vs_pred_{name}")

# Save eval records to CSV
eval_df = pd.DataFrame(eval_records).sort_values("RMSE")
eval_df.to_csv(f"{model_results_path}holdout_performance.csv", index=False)
print(f"\nSaved holdout performance to {model_results_path}holdout_performance.csv")
display(eval_df.round(4))


Evaluating on holdout: Linear
Saved: artifacts/model_plots_lag/

Evaluating on holdout: RandomForest
Saved: artifacts/model_plots_lag/

Evaluating on holdout: XGBoost
Saved: artifacts/model_plots_lag/

Evaluating on holdout: LightGBM
Saved: artifacts/model_plots_lag/

Saved holdout performance to artifacts/model_results_lag/holdout_performance.csv


,RMSE,MSE,MAE,MAPE,R2,Model
3,1.5462,2.3907,0.9238,1.8708,0.9916,LightGBM
2,2.7920,7.7952,1.8287,3.6379,0.9727,XGBoost
1,3.0813,9.4942,1.9003,3.6627,0.9668,RandomForest
0,3.6116,13.0433,2.2207,4.4016,0.9543,Linear


In [ ]:
cv_df = pd.DataFrame(cv_summary)
cv_df.to_csv(f"{model_results_path}cv_summary.csv", index=False)
print(f"Saved CV summary to {model_results_path}cv_summary.csv")
display(cv_df.round(4))

# Plot CV RMSE comparison
if "RMSE_mean" in cv_df.columns:
    fig, ax = plt.subplots(figsize=(8,4))
    sns.barplot(x="Model", y="RMSE_mean", data=cv_df, dodge=False, ax=ax)
    ax.set_title("CV: Mean RMSE by Model")
    save_plot(fig, "cv_rmse_by_model")

# Plot holdout RMSE
fig, ax = plt.subplots(figsize=(8,4))
sns.barplot(x="Model", y="RMSE", data=eval_df.rename(columns={"RMSE":"RMSE"}), dodge=False, ax=ax)
ax.set_title("Holdout: RMSE by Model")
save_plot(fig, "holdout_rmse_by_model")




Saved CV summary to artifacts/model_results_lag/cv_summary.csv


,Model,RMSE_mean,MSE_mean,MAE_mean,MAPE_mean,R2_mean,RMSE_std
0,Linear,10.8298,185.2598,8.9272,16.2834,-0.7448,NaN
1,RandomForest,7.1370,60.6909,4.7987,8.5182,0.7635,0.0
2,XGBoost,7.6832,74.7722,5.2499,8.8160,0.7923,0.0
3,LightGBM,8.3460,87.6943,5.5336,9.5036,0.7230,0.0


Saved: artifacts/model_plots_lag/
Saved: artifacts/model_plots_lag/


In [ ]:
# --------- 10. Feature importance for tree models (Permutation or built-in) ---------
# If RandomForest present, extract feature importances (via fitted pipeline)
if "RandomForest" in best_models:
    rf = best_models["RandomForest"]
    try:
        # Get feature names after preprocessing
        pre = rf.named_steps["preproc"]
        # numeric names and onehot names
        num_names = numeric_cols
        # get onehot feature names if any categories
        try:
            ohe = pre.named_transformers_["cat"].named_steps["onehot"]
            ohe_names = list(ohe.get_feature_names_out(cat_cols))
        except Exception:
            ohe_names = []
        feat_names = num_names + ohe_names

        # Extract underlying RandomForest
        rf_est = rf.named_steps["est"]
        importances = getattr(rf_est, "feature_importances_", None)
        if importances is not None:
            imp_df = pd.DataFrame({"feature": feat_names, "importance": importances})
            imp_df = imp_df.sort_values("importance", ascending=False).head(30)
            imp_df.to_csv("artifacts/model_results/rf_feature_importances.csv", index=False)

            fig, ax = plt.subplots(figsize=(8,6))
            sns.barplot(x="importance", y="feature", data=imp_df, ax=ax)
            ax.set_title("RandomForest: Top feature importances")
            save_plot(fig, "rf_top_feature_importances")
    except Exception as e:
        print("Failed to extract RF feature importances:", e)

Failed to extract RF feature importances: Cannot save file into a non-existent directory: 'artifacts\model_results'


In [ ]:
if "ARIMA" in RUN_MODELS and ARIMA is not None:
    print("\nRunning simple ARIMA baseline on target (univariate) ...")
    try:
        series = df.set_index("Date")[TARGET].asfreq("D").interpolate()
        arima_order = (1,1,1)   # simple baseline
        arima_model = ARIMA(series.iloc[:-n_holdout], order=arima_order).fit()
        arima_fore = arima_model.forecast(steps=n_holdout)
        mets = regression_metrics(series.iloc[-n_holdout:].values, arima_fore)
        mets["Model"] = "ARIMA"
        # append to eval records and save plot
        eval_df = pd.concat([eval_df, pd.DataFrame([mets])], ignore_index=True)
        # Plot
        fig, ax = plt.subplots(figsize=(10,3))
        ax.plot(series.index[-n_holdout:], series.iloc[-n_holdout:], label="Actual")
        ax.plot(series.index[-n_holdout:], arima_fore, linestyle="--", label="ARIMA_Forecast")
        ax.set_title("ARIMA: Actual vs Forecast (holdout)")
        ax.legend()
        save_plot(fig, "arima_holdout")
        # Save ARIMA model
        joblib.dump(arima_model, "artifacts/models/arima_baseline.joblib")
        print("Saved ARIMA baseline model.")
    except Exception as e:
        print("ARIMA failed:", e)


Running simple ARIMA baseline on target (univariate) ...
Saved: artifacts/model_plots_lag/
ARIMA failed: [Errno 2] No such file or directory: 'artifacts/models/arima_baseline.joblib'


In [ ]:
manifest = {
    "timestamp": datetime.utcnow().isoformat(),
    "input_csv": INPUT_CSV,
    "target": TARGET,
    "features_used": FEATURES_TO_USE,
    "numeric_cols": numeric_cols,
    "cat_cols": cat_cols,
    "models_trained": list(best_models.keys()),
    "holdout_rows": int(n_holdout)
}
with open(f"{model_results_path}manifest.json", "w") as f:
    json.dump(manifest, f, indent=2)
print(f"Saved manifest -> {model_results_path}manifest.json")

print("\nNotebook 2 complete. Artifacts saved under artifacts/model_plots, artifacts/models, artifacts/model_results.")

Saved manifest -> artifacts/model_results_lag/manifest.json

Notebook 2 complete. Artifacts saved under artifacts/model_plots, artifacts/models, artifacts/model_results.
